<a href="https://colab.research.google.com/github/Pratush613/summaryApp/blob/main/runStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install streamlit -q
!pip install transformers
!pip install newspaper3k
!pip install fpdf

In [28]:
! wget -q -O - ipv4.icanhazip.com

104.198.105.121


In [31]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
from newspaper import Article
from fpdf import FPDF, HTMLMixin
import base64

class MyFPDF(FPDF, HTMLMixin):
    pass

def convert_to_pdf(text):
    pdf = MyFPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Split text into lines to add to PDF
    lines = text.split('\n')
    for line in lines:
        pdf.multi_cell(0, 10, line.encode('latin-1', 'replace').decode('latin-1'))

    pdf_output = "output.pdf"
    pdf.output(pdf_output)
    return pdf_output

def convert_to_text(text):
    text_file = "output.txt"
    with open(text_file, "w", encoding='utf-8') as file:
        file.write(text)
    return text_file

def get_binary_file_downloader_html(bin_file, file_label):
    with open(bin_file, 'rb') as f:
        data = f.read()
    bin_str = base64.b64encode(data).decode()
    href = f'<a href="data:application/octet-stream;base64,{bin_str}" download="{bin_file}">Download {file_label}</a>'
    return href

st.markdown(
    """
    <style>
    .reportview-container {
        background-color: #032c40;
    }
    footer {
        visibility: hidden;
    }
    .custom-footer {
        position: fixed;
        left: 0;
        bottom: 0;
        width: 100%;
        background-color: #808080;
        text-align: center;
        padding: 10px;
        font-size: 16px;
        color:  #FFFFFF;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

st.sidebar.title("Article Summarizer and Translator")

task = st.sidebar.radio("Choose The Task  ", ["Summarization", "Translation"])

if task == "Summarization":
    st.title("Article Summarizer")

    # Load the summarization pipeline
    summarization_pipeline = pipeline("summarization", model="t5-small")

    input_type = st.radio("Summarize from:", ["Text Input", "URL"])
    max_length = st.slider("Maximum Summary Length:", min_value=50, max_value=500, value=300)
    min_length = st.slider("Minimum Summary Length:", min_value=30, max_value=300, value=100)

    if input_type == "Text Input":
        input_text = st.text_area("Enter text to summarize:", height=150)
        if st.button("Summarize"):
            if input_text:
                try:
                    with st.spinner('Summarizing...'):
                        summary = summarization_pipeline(input_text, max_length=max_length, min_length=min_length)[0]["summary_text"]
                    st.write("Summary:")
                    st.write(summary)

                    pdf_file = convert_to_pdf(summary)
                    text_file = convert_to_text(summary)
                    st.markdown(get_binary_file_downloader_html(pdf_file, "Summary as PDF"), unsafe_allow_html=True)
                    st.markdown(get_binary_file_downloader_html(text_file, "Summary as Text"), unsafe_allow_html=True)
                except Exception as e:
                    st.error("Error summarizing the text. Please try again.")
                    st.error(str(e))

    elif input_type == "URL":
        url = st.text_input("Enter URL to summarize:")
        if st.button("Fetch and Summarize"):
            if url and url.startswith(("http://", "https://")):
                try:
                    article = Article(url)
                    article.download()
                    article.parse()
                    input_text = article.text
                    with st.spinner('Summarizing...'):
                        summary = summarization_pipeline(input_text, max_length=max_length, min_length=min_length)[0]["summary_text"]
                    st.write("Summary:")
                    st.write(summary)

                    pdf_file = convert_to_pdf(summary)
                    text_file = convert_to_text(summary)
                    st.markdown(get_binary_file_downloader_html(pdf_file, "Summary as PDF"), unsafe_allow_html=True)
                    st.markdown(get_binary_file_downloader_html(text_file, "Summary as Text"), unsafe_allow_html=True)
                except Exception as e:
                    st.error("Error fetching or summarizing the article. It might be protected against scraping or is not valid. Please try another URL.")
                    st.error(str(e))
            else:
                st.warning("Please enter a valid URL (starting with http:// or https://).")

elif task == "Translation":
    st.title("Text Translator")

    source_lang = st.selectbox('Select the source language', ['en', 'fr', 'hi'])
    target_lang = st.selectbox('Select the target language', ['en', 'fr', 'hi'])

    model_map = {
        ('en', 'fr'): "Helsinki-NLP/opus-mt-en-fr",
        ('en', 'hi'): "Helsinki-NLP/opus-mt-en-hi",
        ('fr', 'en'): "Helsinki-NLP/opus-mt-fr-en",
        ('hi', 'en'): "Helsinki-NLP/opus-mt-hi-en",
        ('hi', 'fr'): "Helsinki-NLP/opus-mt-hi-fr",
        ('fr', 'hi'): "Helsinki-NLP/opus-mt-fr-hi"
    }

    model_name = model_map.get((source_lang, target_lang), "Helsinki-NLP/opus-mt-en-hi")

    st.write("Source Language:", source_lang)
    st.write("Target Language:", target_lang)

    try:
        translation_pipeline = pipeline("translation", model=model_name)
    except Exception as e:
        st.error(f"Error loading translation model: {str(e)}")
        st.stop()

    input_text = st.text_area("Enter text to translate:", height=150)
    if st.button("Translate"):
        if input_text:
            try:
                with st.spinner('Translating...'):
                    translation = translation_pipeline(input_text)[0]["translation_text"]
                st.write("Translation:")
                st.write(translation)

                pdf_file = convert_to_pdf(translation)
                text_file = convert_to_text(translation)
                st.markdown(get_binary_file_downloader_html(pdf_file, "Translation as PDF"), unsafe_allow_html=True)
                st.markdown(get_binary_file_downloader_html(text_file, "Translation as Text"), unsafe_allow_html=True)
            except Exception as e:
                st.error("Error translating the text. Please try again.")
                st.error(str(e))

# Footer content with name
st.markdown(
    """
    <div class="custom-footer">
        @created by Pratush
    </div>
    """,
    unsafe_allow_html=True,
)


Overwriting app.py


In [32]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.198.105.121:8501

npx: installed 22 in 3.1s
your url is: https://slimy-mangos-speak.loca.lt
2024-06-25 09:41:33.859176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 09:41:33.859298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 09:41:33.861849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-25 09:41:35.736782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin